In [1]:
import pypsa

import pandas as pd
import matplotlib.pyplot as plt

# Demand

In [ ]:
n2025 = pypsa.Network(
    "networks/base_s_50_lc1.5__Ca-Ia-Ea-Tb_2025.nc"
)
n2040 = pypsa.Network(
    "networks/base_s_50_lc1.5__Ca-Ia-Ea-Tb_2040.nc"
)
n2050 = pypsa.Network(
    "networks/base_s_50_lc1.5__Ca-Ia-Ea-Tb_2050.nc"
)

eb25 = n2025.statistics.energy_balance()
eb50 = n2050.statistics.energy_balance()

INFO:pypsa.io:Imported network base_s_50_lc1.5__Ca-Ia-Ea-Tb_2025.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_50_lc1.5__Ca-Ia-Ea-Tb_2040.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_50_lc1.5__Ca-Ia-Ea-Tb_2050.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [3]:
# Ammonia demand
nh3_tonnes = -eb25.loc[("Load", "NH3")].iloc[0] / 1e6

print("Ammonia demand [Mt/a]:", round(nh3_tonnes / 5.18, 1))
print("Ammonia demand [TWh]:", round(nh3_tonnes, 1))

Ammonia demand [Mt/a]: 16.5
Ammonia demand [TWh]: 85.4


In [4]:
# Methanol demand
meoh = (
    -eb25.xs("industry methanol", level="bus_carrier")
    .xs("Load", level="component")
    .squeeze()
    / 1e6
)

print("Methanol demand [TWh/a]:", round(meoh, 1))

Methanol demand [TWh/a]: 8.7


In [5]:
# Oil demand in industry
eb25.xs("oil", level="bus_carrier")
oil_ind_25 = -eb25.loc[("Link", "naphtha for industry", "oil")] / 1e6
oil_ind_50 = -eb50.loc[("Link", "naphtha for industry", "oil")] / 1e6

print(
    f"Oil demand for industry [Mt/a]: {round(oil_ind_25, 1)} (2025), {round(oil_ind_50, 1)} (2050)"
)

Oil demand for industry [Mt/a]: 918.7 (2025), 276.8 (2050)


In [6]:
# Agriculture machinery oil demand
oil_agg = (
    -eb50.xs("oil", level="bus_carrier")
    .xs("agriculture machinery oil", level="carrier")
    .squeeze()
    / 1e6
)

print(f"Oil demand for agriculture machinery [TWh/a]: {round(oil_agg, 1)}")

Oil demand for agriculture machinery [TWh/a]: 102.8


# Demand of EU vs. modelling region

In [49]:
# Modelling region:
modelling_region = [
    "AL",
    "AT",
    "BA",
    "BE",
    "BG",
    "CH",
    "CZ",
    "DE",
    "DK",
    "EE",
    "ES",
    "FI",
    "FR",
    "GB",
    "GR",
    "HR",
    "HU",
    "IE",
    "IT",
    "LT",
    "LU",
    "LV",
    "ME",
    "MK",
    "NL",
    "NO",
    "PL",
    "PT",
    "RO",
    "RS",
    "SE",
    "SI",
    "SK",
]
# EU countries
EU_countries = [
    "AT",
    "BE",
    "BG",
    "CY",
    "CZ",
    "DE",
    "DK",
    "EE",
    "ES",
    "FI",
    "FR",
    "GR",
    "HR",
    "HU",
    "IE",
    "IT",
    "LT",
    "LU",
    "LV",
    "MT",
    "NL",
    "PL",
    "PT",
    "RO",
    "SE",
    "SI",
    "SK",
]

# EU countries with modelling region
EU_modelling_region = [
    country for country in EU_countries if country in modelling_region
]

total_load = (
    (n2040.snapshot_weightings.objective @ n2040.loads_t.p_set).rename("p_set")
    + n2040.loads.p_set
).fillna(0)

# Extract total load by country: index of total_load starts with two-letter country codes
total_load_by_country = total_load.groupby(total_load.index.str[:2], axis=0).sum()
total_load_by_country = total_load_by_country.loc[
    total_load_by_country.index.isin(modelling_region)
]

# Fraction of EU total load
total_load_EU = total_load_by_country.loc[
    total_load_by_country.index.isin(EU_modelling_region)
]
EU_frac = total_load_EU.sum() / total_load_by_country.sum()
print(
    f"Fraction of EU total load in modelling region: {round(EU_frac * 100, 1)} %"
)

Fraction of EU total load in modelling region: 82.9 %


# Supply

In [ ]:
# Biomass availability
# Colunms: multiindex with (setting, solid/gas)
biomass = pd.DataFrame(
    index=["2025", "2030", "2035", "2040", "2045", "2050"],
    columns=pd.MultiIndex.from_product([["Solid biomass", "Biogas"], ["a", "b", "c"]]),
    dtype=float,
)
for y in biomass.index:
    for i in ["a", "b", "c"]:
        df = pd.read_csv(
            f"../resources/Y1987_B{i}/biomass_potentials_s_50_{y}.csv"
        )
        biomass.at[y, ("Solid biomass", i)] = df["solid biomass"].sum() / 1e6
        biomass.at[y, ("Biogas", i)] = df["biogas"].sum() / 1e6

biomass = biomass.round(0).astype(int)

display(biomass)

# Export as csv without header
biomass.to_csv("figures/biomass_potentials.csv", index_label="Year", header=False)

Solid biomass             Biogas          
                 a     b     c      a    b    c
2025           217   382   869     56  113  170
2030           366   688  1733    114  228  342
2035           535  1027  2574    174  348  522
2040           521  1020  2541    177  351  525
2045           427  1017  2585    179  354  529
2050           336  1014  2628    181  358  533

In [8]:
# Total installable capacity in TW
gen_ex = n2025.generators.loc[n2025.generators.p_nom_extendable]
print((gen_ex.groupby("carrier").p_nom_max.sum()[["solar", "onwind"]] / 1e6).round(1))

carrier
solar     10.7
onwind     8.6
Name: p_nom_max, dtype: float64


# Costs

In [9]:
years = [2025, 2030, 2035, 2040, 2045, 2050]

import_carriers = {
    "H2": "shipping-lh2",
    "ammonia": "shipping-lnh3",
    "methanol": "shipping-meoh",
    "oil": "shipping-ftfuel",
    "gas": "shipping-lch4",
}

pretty_carrier_names = {
    "H2": "Hydrogen",
    "ammonia": "Ammonia",
    "methanol": "Methanol",
    "oil": "Oil",
    "gas": "Gas",
}

costs = {
    y: pd.read_csv(f"../resources/Y1987_Bb/costs_{y}.csv", index_col=[0, 1])
    for y in years
}

# Create table with years as columns, carriers as rows and fuel costs as values
import_costs = pd.DataFrame(index=import_carriers.keys(), columns=years)
for y in years:
    for carrier, name in import_carriers.items():
        import_costs.at[carrier, y] = costs[y].at[(name, "fuel"), "value"]

import_costs.index = import_costs.index.map(pretty_carrier_names)
# Round values to 1 decimal
import_costs = import_costs.astype(float).round(1)

import_costs.index.name = "Carrier"

display(import_costs)
import_costs.to_csv("figures/import_costs.csv")

,2025,2030,2035,2040,2045,2050
Carrier,,,,,,
Hydrogen,122.9,122.9,112.0,101.2,92.7,84.1
Ammonia,120.1,120.1,110.3,100.6,94.5,88.4
Methanol,165.6,165.6,151.9,138.2,128.5,118.7
Oil,234.2,234.2,214.0,193.9,178.6,163.4
Gas,128.4,128.4,118.2,108.1,102.2,96.2


In [67]:
electrolysis_costs = {
    h: costs[h].loc[("electrolysis", "investment"), "value"] for h in years
}
electrolysis_costs = pd.Series(electrolysis_costs)

display(electrolysis_costs.round(0))

2025    2152.0
2030    1793.0
2035    1614.0
2040    1435.0
2045    1315.0
2050    1196.0
dtype: float64

In [69]:
electrolysis_efficiency = {
    h: costs[h].loc[("electrolysis", "efficiency"), "value"] for h in years
}
electrolysis_efficiency = pd.Series(electrolysis_efficiency)

display(electrolysis_efficiency.round(3))

2025    0.587
2030    0.622
2035    0.637
2040    0.653
2045    0.676
2050    0.699
dtype: float64

# Conversion technologies

In [10]:
n = n2050

for carrier in ["oil", "gas", "methanol", "NH3", "H2"]:
    print(
        f"Production of {carrier}: {n.links.loc[n.links.bus1.map(n.buses.carrier) == carrier].carrier.unique()}"
    )

Production of oil: ['oil green import' 'oil refining' 'Fischer-Tropsch'
 'biomass to liquid CC' 'biomass to liquid']
Production of gas: ['biogas to gas' 'gas green import' 'gas pipeline' 'Sabatier'
 'gas pipeline new']
Production of methanol: ['methanol green import' 'methanolisation' 'biomass-to-methanol CC'
 'biomass-to-methanol']
Production of NH3: ['ammonia green import' 'Haber-Bosch']
Production of H2: ['H2 Electrolysis' 'H2 green import' 'solid biomass to hydrogen' 'SMR CC'
 'SMR' 'ammonia cracker' 'H2 pipeline']


# Storage technologies

In [11]:
n.stores.carrier.groupby(n.stores.bus.map(n.buses.carrier)).unique()

bus
EV battery                                      [EV battery]
H2                                                [H2 Store]
NH3                                          [ammonia store]
battery                                            [battery]
biogas                                              [biogas]
co2                                                    [co2]
co2 sequestered                            [co2 sequestered]
co2 stored                                      [co2 stored]
coal                                                  [coal]
gas                                                    [gas]
lignite                                            [lignite]
methanol                                          [methanol]
oil                                                    [oil]
rural water tanks                        [rural water tanks]
solid biomass                                [solid biomass]
uranium                                            [uranium]
urban central water 

In [12]:
print("Dispatch capacity:")
display(n.storage_units.groupby("carrier").p_nom.sum() / 1e3) # GW

print("Max hours:")
display(n.storage_units.groupby("carrier").max_hours.mean())

print("Storage capacity:")
display(
    (n.storage_units.p_nom * n.storage_units.max_hours)
    .groupby(n.storage_units.carrier)
    .sum()
    / 1e6 # TWh
)

Dispatch capacity:


carrier
PHS       56.87754
hydro    103.14744
Name: p_nom, dtype: float64

Max hours:


carrier
PHS       148.395521
hydro    8510.298593
Name: max_hours, dtype: float64

Storage capacity:


carrier
PHS        8.660784
hydro    149.284524
dtype: float64

In [13]:
# BEV storage
bev_battery = n.stores.loc[n.stores.carrier == "EV battery"].index
print("EV battery capacity:", n.stores.loc[bev_battery].e_nom.sum() / 1e6) # TWh

# BEV v2g capacity
print("EV v2g dispatch capacity:", n.links.loc[n.links.carrier == "V2G", "p_nom"].sum() / 1e3) # GW

EV battery capacity: 5.217019242860411
EV v2g dispatch capacity: 2295.488466858581


In [14]:
# Salt cavern hydrogen storage
salt_caverns = n.stores.loc[(n.stores.carrier == "H2 Store") & (n.stores.index.str.endswith("-2050"))]

print("Salt cavern locations:" , salt_caverns.bus.unique())

print("Max potential underground hydrogen storage:", salt_caverns.e_nom_max.sum() / 1e6) # TWh

Salt cavern locations: ['DE0 0 H2' 'DE0 3 H2' 'DE0 5 H2' 'DK0 0 H2' 'DK1 0 H2' 'ES0 0 H2'
 'GB3 0 H2' 'GB3 2 H2' 'GB4 0 H2' 'GR0 0 H2' 'NL0 0 H2' 'PL0 0 H2'
 'PT0 0 H2']
Max potential underground hydrogen storage: 2559.9107659891033


In [15]:
# Gas storage
gas_storage = n2025.stores.loc[n2025.stores.carrier == "gas"]
print("Gas storage capacity:", gas_storage.e_nom_opt.sum() / 1e6) # TWh

Gas storage capacity: 1396.904749488332


# Emissions

In [16]:
print("Process emissions in 2025:", n2025.statistics.energy_balance().loc[("Load", "process emissions")].squeeze() / 1e6)
print("Process emissions in 2050:", n2050.statistics.energy_balance().loc[("Load", "process emissions")].squeeze() / 1e6)

Process emissions in 2025: 189.85
Process emissions in 2050: 123.07


# Value of hydrogen sector

In [19]:
fn = f"cache_results.csv"
cols = pd.read_csv(fn).columns
h2prod_col = cols.get_loc("h2prod")
df = pd.read_csv(fn, index_col=list(range(h2prod_col)))

# Convert "year" index level values from int to str
df.index = df.index.set_levels(df.index.levels[1].astype(str), level=1)

In [20]:
frac_sys_cost = (df.h2value / df.system_cost).xs("opt", level="sense").xs(2050, level="horizon")

print("Fraction of system cost covered by hydrogen value:")
display((frac_sys_cost * 100).describe().round(1))

Fraction of system cost covered by hydrogen value:


count    216.0
mean       3.9
std        3.8
min        0.0
25%        0.1
50%        3.3
75%        6.9
max       15.9
dtype: float64